In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import random
from IPython.display import display, HTML
from IPython.display import display, Markdown

import pandas as pd
import base64  
import numpy as np

# Code 3
## Used to verify the output of the final report by calculation rates based on detailed proposal output

In [2]:

# temp = 'temp_'
uat  = 'uat_'
file_Index = 1


In [3]:
file_name = 'testnewformat.csv'
SHOW_ALL_OUTPUT = False


In [4]:
def calFee(file_name):
    df_UAT = pd.read_csv(file_name, names = ['name','rate_type','rate','best','lowest','highest','worst','overrun'])
    df_UAT = df_UAT.dropna()
    df_UAT = df_UAT[df_UAT.name != 'Name']
    dfoverrun = pd.read_csv("overrun.csv",names=['overrun','factor'])
    dfoverrun = dfoverrun.set_index('overrun')

    look_up_dict = dfoverrun.to_dict()['factor']
    df_UAT.overrun = df_UAT.overrun.str.strip()
    df_UAT.overrun = df_UAT.overrun.map(look_up_dict)
    
    df_UAT.best=pd.to_numeric(df_UAT['best'])
    df_UAT.highest=pd.to_numeric(df_UAT['highest'])
    df_UAT.lowest=pd.to_numeric(df_UAT['lowest'])
    df_UAT.worst=pd.to_numeric(df_UAT['worst'])
    df_UAT.rate=pd.to_numeric(df_UAT['rate'])
    df_UAT.overrun=pd.to_numeric(df_UAT['overrun'])
    
    df_UAT['bc']=np.around(df_UAT['best'] * df_UAT['rate'],-1)
    df_UAT['la']=np.around(df_UAT['lowest'] * df_UAT['rate'],-1)
    df_UAT['ha']=np.around(df_UAT['highest'] * df_UAT['rate'],-1)
    df_UAT['wc']=np.around(df_UAT['worst'] * df_UAT['rate'],-1)
    df_UAT['LRFF'] = np.around((df_UAT['wc']-(df_UAT['ha']+df_UAT['la'])*0.5)*df_UAT['overrun']+(df_UAT['ha']+df_UAT['la'])*0.5,-1)
    df_UAT['HRFF'] = np.around((df_UAT['wc']-df_UAT['ha'])*df_UAT['overrun']+df_UAT['ha'],-1)
    sums = df_UAT.sum()
    #display(HTML('<h1>Check output for: <i>'+file_name+'</i></h1>'))
    return ('<h2>Check output for: <i>'+file_name+'</i></h2>'+
    '<h3>Final Fixed Fee: '+str(sums.LRFF)+'</h3>'+
    '<h3>Best Case Cost: '+str(sums.bc)+'</h3>'+
    '<h3>Lowest Anticipated Cost: '+str(sums.la)+'</h3>'+
    '<h3>Highest Anticipated Cost: '+str(sums.ha)+'</h3>'+
    '<h3>Worst Case Cost: '+str(sums.wc)+'</h3>'+
    '<hr>')


In [5]:
def getfiles(Index,Onefile,Quickoutput):
    bigfile = 'ReallyBigFile.html'
    test = Index.split(',')
    stripped = []
    for word1 in test:
        stripped.append(word1.strip())
    stripped
    if Onefile :
        with open(bigfile,'w') as file:
            file.write('<h1>All outputs in one file:</h1>')
    
    for word in stripped:
        file_name = 'input' + word + '.csv'
        output_file = 'input' + word +'.html'
        #print(f'Using file: {file_name} as testing file')
        try:
            output = calFee(file_name)
            
            if Quickoutput :
                display(HTML(output))
            if Onefile == False:
                with open(output_file,'w') as file:
                    file.write(output)
            else:
                with open(bigfile,'a') as file:
                    file.write(output)
        except FileNotFoundError:
            display(HTML('<h1>Incorrect File Name: ' + file_name+'</h1>'))
        
        

interact_manual(getfiles, Index="1",Onefile = False,Quickoutput = True)


interactive(children=(Text(value='1', description='Index'), Checkbox(value=False, description='Onefile'), Chec…

<function __main__.getfiles(Index, Onefile, Quickoutput)>